In [ ]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS 
import datetime
import os
import matplotlib.pyplot as plt
import seaborn as sn
import plotly.graph_objects as go
from geopy.geocoders import Nominatim
import math
import requests
import io
import sklearn
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',1000)
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df=pd.read_csv('/content/drive/My Drive/Copy of US_Accidents_June20.csv') #,nrows=10000)
redunCol=['ID','Source','End_Lat','End_Lng','Number','Street','Zipcode','Country','Weather_Timestamp'
            ,'Airport_Code','Astronomical_Twilight','Civil_Twilight','Nautical_Twilight']
df=df.drop(redunCol,axis=1)
df

,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Side,City,County,State,Timezone,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset
0,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,0.010,Right lane blocked due to accident on I-70 Eas...,R,Dayton,Montgomery,OH,US/Eastern,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
1,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,0.010,Accident on Brice Rd at Tussing Rd. Expect del...,L,Reynoldsburg,Franklin,OH,US/Eastern,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
2,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.010,Accident on OH-32 State Route 32 Westbound at ...,R,Williamsburg,Clermont,OH,US/Eastern,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night
3,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.010,Accident on I-75 Southbound at Exits 52 52B US...,R,Dayton,Montgomery,OH,US/Eastern,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
4,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.010,Accident on McEwen Rd at OH-725 Miamisburg Cen...,R,Dayton,Montgomery,OH,US/Eastern,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513612,NaN,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379360,0.543,At Market St - Accident.,R,Riverside,Riverside,CA,US/Pacific,86.0,86.0,40.0,28.92,10.0,W,13.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day
3513613,NaN,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148060,0.338,At Camino Del Rio/Mission Center Rd - Accident.,R,San Diego,San Diego,CA,US/Pacific,70.0,70.0,73.0,29.39,10.0,SW,6.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day
3513614,NaN,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847790,0.561,At Glassell St/Grand Ave - Accident. in the ri...,R,Orange,Orange,CA,US/Pacific,73.0,73.0,64.0,29.74,10.0,SSW,10.0,0.00,Partly Cloudy,False,False,False,False,True,False,False,False,False,False,False,False,False,Day
3513615,NaN,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.992460,-118.403020,0.772,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,R,Culver City,Los Angeles,CA,US/Pacific,71.0,71.0,81.0,29.62,10.0,SW,8.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day


## Data Cleaning

In [ ]:
redunCol=['TMC','Start_Time','End_Time','Description','City','County','State','Timezone','Distance(mi)','Wind_Direction']
df=df.drop(redunCol,axis=1)

In [ ]:
df.loc[df['Side']=='R','Side']=1 #Right side is 1
df.loc[df['Side']=='L','Side']=0 #Left side is 
df.loc[df['Sunrise_Sunset']=='Day','Sunrise_Sunset']=0 #daytime is 0
df.loc[df['Sunrise_Sunset']=='Night','Sunrise_Sunset']=1 #nighttime is 1
df

,Severity,Start_Lat,Start_Lng,Side,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset
0,3,39.865147,-84.058723,1,36.9,NaN,91.0,29.68,10.0,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,1
1,2,39.928059,-82.831184,0,37.9,NaN,100.0,29.65,10.0,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,1
2,2,39.063148,-84.032608,1,36.0,33.3,100.0,29.67,10.0,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,1
3,3,39.747753,-84.205582,1,35.1,31.0,96.0,29.64,9.0,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,1
4,2,39.627781,-84.188354,1,36.0,33.3,89.0,29.65,6.0,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513612,2,34.002480,-117.379360,1,86.0,86.0,40.0,28.92,10.0,13.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0
3513613,2,32.766960,-117.148060,1,70.0,70.0,73.0,29.39,10.0,6.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0
3513614,2,33.775450,-117.847790,1,73.0,73.0,64.0,29.74,10.0,10.0,0.00,Partly Cloudy,False,False,False,False,True,False,False,False,False,False,False,False,False,0
3513615,2,33.992460,-118.403020,1,71.0,71.0,81.0,29.62,10.0,8.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0


In [ ]:
cols=df['Weather_Condition'].unique()
for col in cols:
    df[col] = df['Weather_Condition'] == col

In [ ]:
del(df['Wind_Chill(F)'])


In [ ]:
df["Precipitation(in)"].fillna(0, inplace = True)

In [ ]:
df['Wind_Speed(mph)'].fillna((df['Wind_Speed(mph)'].mean()), inplace = False)
df


,Severity,Start_Lat,Start_Lng,Side,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,...,Partial Fog / Windy,Light Snow with Thunder,Widespread Dust / Windy,Sand / Dust Whirlwinds / Windy,Tornado,Snow and Thunder,Snow and Sleet / Windy,Heavy Snow with Thunder,Thunder / Wintry Mix / Windy,Light Snow Showers,Heavy Blowing Snow,Light Hail,Heavy Smoke,Heavy Thunderstorms with Small Hail,Light Thunderstorm,Heavy Freezing Drizzle,Light Blowing Snow,Thunderstorms and Snow,Freezing Rain / Windy,Dust Whirls,Sand / Dust Whirls Nearby,Heavy Rain Shower,Thunder and Hail,Drifting Snow,Thunder and Hail / Windy
0,3,39.865147,-84.058723,1,36.9,91.0,29.68,10.0,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,1,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2,39.928059,-82.831184,0,37.9,100.0,29.65,10.0,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,1,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2,39.063148,-84.032608,1,36.0,100.0,29.67,10.0,3.5,0.00,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,1,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,3,39.747753,-84.205582,1,35.1,96.0,29.64,9.0,4.6,0.00,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,1,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,2,39.627781,-84.188354,1,36.0,89.0,29.65,6.0,3.5,0.00,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,0,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513612,2,34.002480,-117.379360,1,86.0,40.0,28.92,10.0,13.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513613,2,32.766960,-117.148060,1,70.0,73.0,29.39,10.0,6.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513614,2,33.775450,-117.847790,1,73.0,64.0,29.74,10.0,10.0,0.00,Partly Cloudy,False,False,False,False,True,False,False,False,False,False,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513615,2,33.992460,-118.403020,1,71.0,81.0,29.62,10.0,8.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,0,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
df = df.dropna()

In [ ]:
del(df['Weather_Condition'])

## Splitting into features and severity

In [ ]:
y = df['Severity'].copy()
y

2          2
3          3
4          2
5          3
6          2
          ..
3513612    2
3513613    2
3513614    2
3513615    2
3513616    2
Name: Severity, Length: 3027906, dtype: int64

In [ ]:
x = df.drop(['Severity'], axis = 1)
x

,Start_Lat,Start_Lng,Side,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Light Rain,Overcast,...,Partial Fog / Windy,Light Snow with Thunder,Widespread Dust / Windy,Sand / Dust Whirlwinds / Windy,Tornado,Snow and Thunder,Snow and Sleet / Windy,Heavy Snow with Thunder,Thunder / Wintry Mix / Windy,Light Snow Showers,Heavy Blowing Snow,Light Hail,Heavy Smoke,Heavy Thunderstorms with Small Hail,Light Thunderstorm,Heavy Freezing Drizzle,Light Blowing Snow,Thunderstorms and Snow,Freezing Rain / Windy,Dust Whirls,Sand / Dust Whirls Nearby,Heavy Rain Shower,Thunder and Hail,Drifting Snow,Thunder and Hail / Windy
2,39.063148,-84.032608,1,36.0,100.0,29.67,10.0,3.5,0.00,False,False,False,False,False,False,False,False,False,False,False,True,False,1,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,39.747753,-84.205582,1,35.1,96.0,29.64,9.0,4.6,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,1,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,39.627781,-84.188354,1,36.0,89.0,29.65,6.0,3.5,0.00,False,False,False,False,False,False,False,False,False,False,False,True,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,40.100590,-82.925194,1,37.9,97.0,29.63,7.0,3.5,0.03,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,39.758274,-84.230507,1,34.0,100.0,29.66,7.0,3.5,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513612,34.002480,-117.379360,1,86.0,40.0,28.92,10.0,13.0,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513613,32.766960,-117.148060,1,70.0,73.0,29.39,10.0,6.0,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513614,33.775450,-117.847790,1,73.0,64.0,29.74,10.0,10.0,0.00,False,False,False,False,True,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513615,33.992460,-118.403020,1,71.0,81.0,29.62,10.0,8.0,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## Normalising features

In [ ]:
x['Start_Lat']=(x['Start_Lat']-x['Start_Lat'].mean())/(x['Start_Lat'].std())
x['Start_Lng']=(x['Start_Lng']-x['Start_Lng'].mean())/(x['Start_Lng'].std())
x['Temperature(F)']=(x['Temperature(F)']-x['Temperature(F)'].mean())/(x['Temperature(F)'].std())
x['Humidity(%)']=(x['Humidity(%)']-x['Humidity(%)'].mean())/(x['Humidity(%)'].std())
x['Pressure(in)']=(x['Pressure(in)']-x['Pressure(in)'].mean())/(x['Pressure(in)'].std())
x['Visibility(mi)']=(x['Visibility(mi)']-x['Visibility(mi)'].mean())/(x['Visibility(mi)'].std())
x['Wind_Speed(mph)']=(x['Wind_Speed(mph)']-x['Wind_Speed(mph)'].mean())/(x['Wind_Speed(mph)'].std())

In [ ]:
x

,Start_Lat,Start_Lng,Side,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Light Rain,Overcast,...,Partial Fog / Windy,Light Snow with Thunder,Widespread Dust / Windy,Sand / Dust Whirlwinds / Windy,Tornado,Snow and Thunder,Snow and Sleet / Windy,Heavy Snow with Thunder,Thunder / Wintry Mix / Windy,Light Snow Showers,Heavy Blowing Snow,Light Hail,Heavy Smoke,Heavy Thunderstorms with Small Hail,Light Thunderstorm,Heavy Freezing Drizzle,Light Blowing Snow,Thunderstorms and Snow,Freezing Rain / Windy,Dust Whirls,Sand / Dust Whirls Nearby,Heavy Rain Shower,Thunder and Hail,Drifting Snow,Thunder and Hail / Windy
2,0.504227,0.671577,1,-1.413522,1.584292,-0.040184,0.298710,-0.899843,0.00,False,False,False,False,False,False,False,False,False,False,False,True,False,1,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,0.642589,0.661521,1,-1.461666,1.407573,-0.074843,-0.056632,-0.690148,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,1,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,0.618342,0.662523,1,-1.413522,1.098313,-0.063290,-1.122659,-0.899843,0.00,False,False,False,False,False,False,False,False,False,False,False,True,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,0.713898,0.735956,1,-1.311887,1.451752,-0.086396,-0.767317,-0.899843,0.03,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,0.644715,0.660072,1,-1.520507,1.584292,-0.051737,-0.767317,-0.899843,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513612,-0.518552,-1.267021,1,1.261101,-1.066500,-0.906653,0.298710,0.911158,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513613,-0.768255,-1.253575,1,0.405222,0.391435,-0.363666,0.298710,-0.423264,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513614,-0.564435,-1.294253,1,0.565699,-0.006183,0.040686,0.298710,0.339263,0.00,False,False,False,False,True,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513615,-0.520577,-1.326531,1,0.458714,0.744874,-0.097949,0.298710,-0.042000,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
x.dropna()

,Start_Lat,Start_Lng,Side,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Light Rain,Overcast,...,Partial Fog / Windy,Light Snow with Thunder,Widespread Dust / Windy,Sand / Dust Whirlwinds / Windy,Tornado,Snow and Thunder,Snow and Sleet / Windy,Heavy Snow with Thunder,Thunder / Wintry Mix / Windy,Light Snow Showers,Heavy Blowing Snow,Light Hail,Heavy Smoke,Heavy Thunderstorms with Small Hail,Light Thunderstorm,Heavy Freezing Drizzle,Light Blowing Snow,Thunderstorms and Snow,Freezing Rain / Windy,Dust Whirls,Sand / Dust Whirls Nearby,Heavy Rain Shower,Thunder and Hail,Drifting Snow,Thunder and Hail / Windy
2,0.504227,0.671577,1,-1.413522,1.584292,-0.040184,0.298710,-0.899843,0.00,False,False,False,False,False,False,False,False,False,False,False,True,False,1,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,0.642589,0.661521,1,-1.461666,1.407573,-0.074843,-0.056632,-0.690148,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,1,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,0.618342,0.662523,1,-1.413522,1.098313,-0.063290,-1.122659,-0.899843,0.00,False,False,False,False,False,False,False,False,False,False,False,True,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,0.713898,0.735956,1,-1.311887,1.451752,-0.086396,-0.767317,-0.899843,0.03,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,0.644715,0.660072,1,-1.520507,1.584292,-0.051737,-0.767317,-0.899843,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513612,-0.518552,-1.267021,1,1.261101,-1.066500,-0.906653,0.298710,0.911158,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513613,-0.768255,-1.253575,1,0.405222,0.391435,-0.363666,0.298710,-0.423264,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513614,-0.564435,-1.294253,1,0.565699,-0.006183,0.040686,0.298710,0.339263,0.00,False,False,False,False,True,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513615,-0.520577,-1.326531,1,0.458714,0.744874,-0.097949,0.298710,-0.042000,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
x['Side'] = x['Side'] == 1
x['Sunrise_Sunset'] = x['Sunrise_Sunset'] == 1
x

,Start_Lat,Start_Lng,Side,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Light Rain,Overcast,...,Partial Fog / Windy,Light Snow with Thunder,Widespread Dust / Windy,Sand / Dust Whirlwinds / Windy,Tornado,Snow and Thunder,Snow and Sleet / Windy,Heavy Snow with Thunder,Thunder / Wintry Mix / Windy,Light Snow Showers,Heavy Blowing Snow,Light Hail,Heavy Smoke,Heavy Thunderstorms with Small Hail,Light Thunderstorm,Heavy Freezing Drizzle,Light Blowing Snow,Thunderstorms and Snow,Freezing Rain / Windy,Dust Whirls,Sand / Dust Whirls Nearby,Heavy Rain Shower,Thunder and Hail,Drifting Snow,Thunder and Hail / Windy
2,0.504227,0.671577,True,-1.413522,1.584292,-0.040184,0.298710,-0.899843,0.00,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,0.642589,0.661521,True,-1.461666,1.407573,-0.074843,-0.056632,-0.690148,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,0.618342,0.662523,True,-1.413522,1.098313,-0.063290,-1.122659,-0.899843,0.00,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,0.713898,0.735956,True,-1.311887,1.451752,-0.086396,-0.767317,-0.899843,0.03,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,0.644715,0.660072,True,-1.520507,1.584292,-0.051737,-0.767317,-0.899843,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513612,-0.518552,-1.267021,True,1.261101,-1.066500,-0.906653,0.298710,0.911158,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513613,-0.768255,-1.253575,True,0.405222,0.391435,-0.363666,0.298710,-0.423264,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513614,-0.564435,-1.294253,True,0.565699,-0.006183,0.040686,0.298710,0.339263,0.00,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3513615,-0.520577,-1.326531,True,0.458714,0.744874,-0.097949,0.298710,-0.042000,0.00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
x.dtypes

Start_Lat                              float64
Start_Lng                              float64
Side                                      bool
Temperature(F)                         float64
Humidity(%)                            float64
Pressure(in)                           float64
Visibility(mi)                         float64
Wind_Speed(mph)                        float64
Precipitation(in)                      float64
Amenity                                   bool
Bump                                      bool
Crossing                                  bool
Give_Way                                  bool
Junction                                  bool
No_Exit                                   bool
Railway                                   bool
Roundabout                                bool
Station                                   bool
Stop                                      bool
Traffic_Calming                           bool
Traffic_Signal                            bool
Turning_Loop 

## Splitting into tranning, test and validation datasets

In [ ]:
x_train1, x_test, y_train1, y_test = train_test_split(x,y,test_size=0.1, random_state=42, shuffle=True)

In [ ]:
x_train, x_validation, y_train, y_validation = train_test_split(x_train1,y_train1,test_size=0.167, random_state=42, shuffle=True)

## Data Modeling SVM

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Import DecisionTreeClassifier from sklearn.tree
from sklearn.tree import DecisionTreeClassifier

# Import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

In [ ]:
def batch(iterable_X, iterable_y, n=1):
    l = len(iterable_X)
    for ndx in range(0, l, n):
        yield iterable_X[ndx:min(ndx + n, l)], iterable_y[ndx:min(ndx + n, l)]

In [ ]:
clf = SGDClassifier(alpha=.0001, loss='hinge', penalty='l2', n_jobs=-1, shuffle=True, max_iter=1000, verbose=0, tol=0.001)
 
ROUNDS = 6
for _ in range(ROUNDS):
    batcherator = batch(x_train, y_train, 100000)
    for index, (chunk_X, chunk_y) in enumerate(batcherator):
        clf.partial_fit(chunk_X, chunk_y, classes=[1, 2, 3, 4])
 
        y_predicted = clf.predict(x_test)
        print(accuracy_score(y_test, y_predicted))

0.6499169394070498
0.6716943370179431
0.6687451080117969
0.6797328850593314
0.6798022398287928
0.6793365720909802
0.6796106885607565
0.6797725163561664
0.6797593059238881
0.6797560033158185
0.6797593059238881
0.6797725163561664
0.6797758189642361
0.6797725163561664
0.6797725163561664
0.6797725163561664
0.6797659111400273
0.6797725163561664
0.6797725163561664
0.6797692137480968
0.6797593059238881
0.6795512416155037
0.6797725163561664
0.6797593059238881
0.6797626085319577
0.6797725163561664
0.6797692137480968
0.6797725163561664
0.6797692137480968
0.6797692137480968
0.6797725163561664
0.6797626085319577
0.6797626085319577
0.6797626085319577
0.6797725163561664
0.6797725163561664
0.6797725163561664
0.6797725163561664
0.6797725163561664
0.6797692137480968
0.6797725163561664
0.6797725163561664
0.6797725163561664
0.6797593059238881
0.6797692137480968
0.6797692137480968
0.6797725163561664
0.6797659111400273
0.6797725163561664
0.6797725163561664
0.6797692137480968
0.6797725163561664
0.6797758189

## Classification Matrix

In [ ]:
from sklearn import metrics

print(metrics.classification_report(y_test, y_predicted, digits=3))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



              precision    recall  f1-score   support

           1      0.000     0.000     0.000      2825
           2      0.680     1.000     0.809    205829
           3      0.000     0.000     0.000     84633
           4      0.000     0.000     0.000      9504

    accuracy                          0.680    302791
   macro avg      0.170     0.250     0.202    302791
weighted avg      0.462     0.680     0.550    302791



## Data Modeling logistic regression

In [ ]:
clf = SGDClassifier(alpha=.0001, loss='log', penalty='l2', n_jobs=-1, shuffle=True, max_iter=1000, verbose=0, tol=0.001)
 
ROUNDS = 6
for _ in range(ROUNDS):
    batcherator = batch(x_train, y_train, 100000)
    for index, (chunk_X, chunk_y) in enumerate(batcherator):
        clf.partial_fit(chunk_X, chunk_y, classes=[1, 2, 3, 4])
 
        y_predicted = clf.predict(x_test)
        print(accuracy_score(y_test, y_predicted))

0.6795644520477822
0.6790855738776912
0.6660799032996357
0.6715060883579763
0.6781080018890918
0.666377138025899
0.6756277432288278
0.6782335009957363
0.675456007609209
0.6783259740216849
0.6743000947848516
0.6779329636614034
0.6767076960675845
0.6748516303324735
0.6788808121773765
0.6765392630560354
0.6774937167881476
0.6784944070332342
0.6790954817019
0.6789931008517426
0.6775333480849827
0.679752700707749
0.6767704456209068
0.6794587685895551
0.6791119947422479
0.679485189454112
0.6793861112120242
0.6772196003183715
0.675244640692755
0.6778537010677332
0.6796899511544267
0.6790129165001602
0.6772658368313457
0.6754626128253481
0.6769884177534999
0.6792672173215188
0.6760306614133181
0.6772592316152065
0.6789864956356034
0.6780452523357695
0.6788246678401934
0.6791549286471527
0.6791582312552222
0.6783788157507984
0.6794950972783207
0.6781278175375094
0.6765161447995482
0.6778404906354548
0.6791747442955702
0.6784646835606079
0.678917140866142
0.6787255895981056
0.6763906456929036
0.

## Classification Matrix

In [ ]:
from sklearn import metrics

print(metrics.classification_report(y_test, y_predicted, digits=3))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



              precision    recall  f1-score   support

           1      0.000     0.000     0.000      2825
           2      0.686     0.977     0.806    205829
           3      0.449     0.052     0.093     84633
           4      0.000     0.000     0.000      9504

    accuracy                          0.678    302791
   macro avg      0.284     0.257     0.225    302791
weighted avg      0.592     0.678     0.574    302791

